<a href="https://colab.research.google.com/github/crowmastr/CAP6640/blob/progress/neural%20network/hate_speech_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Imports

In [1]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

import keras
from keras import backend as K
from keras.layers import Activation, Embedding, GlobalAveragePooling1D
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.text import text_to_word_sequence

import numpy as np
from numpy import array

import pandas as pd
import io

Using TensorFlow backend.


### Upload data

In [2]:
from google.colab import files
uploaded = files.upload()

Saving labeled_data.csv to labeled_data.csv


In [4]:
data_frame = pd.read_csv(io.StringIO(uploaded['labeled_data.csv'].decode('utf-8')))
data_frame.columns
data_frame

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


### Display Data

In [5]:
pd.set_option('display.max_colwidth', -1)
display(data_frame)
sorted_by_hate = data_frame.sort_values(by='hate_speech', axis='index', ascending=False)
display(sorted_by_hate)


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,"you's a muthaf***in lie &#8220;@LifeAsKing: @20_Pearls @corey_emanuel right! His TL is trash &#8230;. Now, mine? Bible scriptures and hymns&#8221;"
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, and drove me redneck crazy"
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
6171,6348,9,7,1,1,0,"@infidelpamelaLC I'm going to blame the black man, since they always blame ""whitey"" I'm an equal opportunity hater."
15658,16024,9,7,2,0,0,RT @Isa__Lopez: @D_Lo520 but you're still a faggot
10451,10729,9,7,2,0,0,I hate fat bitches
15809,16178,9,6,0,3,0,"RT @JihadistJoe: We Muslims have no military honour whatsoever, we are sub human savages that slaughter unarmed men, women &amp; children http:&#8230;"
3869,3982,7,6,1,0,0,@L1LTR4P fucking losers wetbacks #SorryNotSorry
...,...,...,...,...,...,...,...
9438,9697,3,0,3,0,1,Girls the days be confused hoes ain't nobody got time fo that !
9437,9696,3,0,0,3,2,Girls that wear colored jeans &#128525;&#128527;&#128076;
9435,9694,4,0,4,0,1,Girls be looking for reasons to do hoe shit
9434,9693,6,0,5,1,1,"Girls be like ""10/23/12 I love you babe &lt;3"" hoe isn't that today?"


### Extract Tweets

In [6]:
tweets = data_frame['tweet']
print(tweets)
labels = data_frame['hate_speech']
print(labels)

0        !!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...      
1        !!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!                                                             
2        !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit                          
3        !!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny                                                                                    
4        !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;         
                                                                           ...                                                                             
24778    you's a muthaf***in lie &#8220;@LifeAsKing: @20_Pearls 

### Calculate how many unique vocab words there are in the Tweets (answer: 40,817)

In [112]:
index = 10000
punctuation = "!\"#$%&()*+,'-./:;<=>?@[\]^_`{|}~"
table = str.maketrans("","",punctuation)
vocab = set()
processed_tweets = list()

for tweet in tweets:
    processed_tweet = tweet.lower().translate(table).split()
    vocab = vocab.union(processed_tweet)
    processed_tweets.append(processed_tweet)

vocab = sorted(vocab)
print(len(vocab))
#vocab = 40817, you don't need to check it every single time, it's the same every time(!)


40817


Calculate max len (answer: 33 )

In [111]:
maxlen = np.max(np.vectorize(len)(processed_tweets))
maxlen
#maxlen = 33

520

### Pre-process the Tweets and labels

Remove most punctuation from Tweets, make them all lowercase, make labels binary (0 or 1) instead of 0 to 7.

Hyperparams

In [0]:
# Embedding
max_features = 41000
maxlen = 33
embedding_size = 128

# Convolution
kernel_size = 5
filters = 54
pool_size = 4

# LSTM
lstm_output_size = 64

# Training
batch_size = 16
epochs = 2

In [124]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

processed_tweets = []

punctuation = "!\"#$%&()*+,'-./:;<=>?@[\]^_`{|}~"
table = str.maketrans("","",punctuation)

for tweet in tweets:
    processed_tweets.append(tweet.lower().translate(table))

encoded_tweets = [one_hot(tweet, max_features) for tweet in processed_tweets]

padded_tweets = pad_sequences(encoded_tweets, dtype='int32', padding='post', value=0, maxlen= maxlen)

binary_labels = []

for label in labels:
    if label > 0:
        binary_labels.append(1)
    else:
        binary_labels.append(0)


print(padded_tweets[0])
# Number of tweets for each label
# unique, counts = np.unique(labels.values, return_counts=True)
# dict(zip(unique, counts))

[ 2168 38333 13483 12834  9889 31259  2196 39271 39609  6666 19274 37484
 17739 30776 13483 12834 39276 31259  8673 22114  9407  6524 15487 19545
     0     0     0     0     0     0     0     0     0]


Break data set in training and test sets

In [0]:


# break data set
x_train = padded_tweets[:20000]
x_test = padded_tweets[20000:]

y_train = binary_labels[:20000]
y_test = binary_labels[20000:]


In [0]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.1))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=2))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [127]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 33, 128)           5248000   
_________________________________________________________________
dropout_23 (Dropout)         (None, 33, 128)           0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 15, 54)            34614     
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 3, 54)             0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 64)                30464     
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 65        
_________________________________________________________________
activation_23 (Activation)   (None, 1)               

In [0]:
#import numpy as np
#np.reshape(tweets, (-1,1))
#processed_tweets.transpose().shape

#processed_tweets = processed_tweets.reshape((24783, 1))
#keras.preprocessing.sequence.pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)


In [128]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 20000 samples, validate on 4783 samples
Epoch 1/2
20000/20000 [==============================] - 27s 1ms/step - loss: 0.4390 - acc: 0.8173 - val_loss: 0.4034 - val_acc: 0.8382
Epoch 2/2
4783/4783 [==============================] - 1s 250us/step
Test score: 0.47448499607299627
Test accuracy: 0.8095337654316548


In [129]:
score = model.evaluate(x_test, y_test, batch_size=32)
print(str(round(score[1]*100,2))+"%")

4783/4783 [==============================] - 1s 143us/step
80.95%
